# SnapLearn - Démo Pipeline Complet

Ce notebook démontre le pipeline complet de SnapLearn:
1. Traitement de documents (PDF/Image)
2. Recherche RAG dans la base de connaissance
3. Génération de script audio
4. Synthèse vocale
5. Quiz interactif

In [ ]:
# Imports
import sys
sys.path.append('..')

from src.universal_document_processor import UniversalDocumentProcessor
from src.amu_knowledge_base import AMUKnowledgeBase
from src.audio_script_generator import AudioScriptGenerator
from src.audio_generator import AudioGenerator

import os
from dotenv import load_dotenv

load_dotenv()

print("Imports réussis")

## 1. Test de la Base de Connaissance RAG

In [ ]:
# Initialiser la base de connaissance
kb = AMUKnowledgeBase('../data/amu_datascience_corpus.json')

# Test de recherche
query = "Qu'est-ce qu'un transformer?"
results = kb.search(query, top_k=3)

print(f"Recherche: '{query}'\n")
for i, result in enumerate(results, 1):
    print(f"Résultat {i} (score: {result['score']:.2f})")
    print(f"Type: {result['metadata']['type']}")
    print(f"Texte: {result['text'][:200]}...")
    print("-" * 80)

## 2. Test du Traitement de Documents

In [ ]:
# Initialiser le processeur
processor = UniversalDocumentProcessor(os.getenv('GEMINI_API_KEY'))

# Test avec un fichier (remplacer par votre fichier de test)
# test_file = '../data/course_materials/m2/s2/week_1_intro.pdf'
# result = processor.process_document(test_file)

# print(f"Type: {result['type']}")
# if result['type'] == 'pdf':
#     print(f"Pages: {result['metadata']['total_pages']}")
#     print(f"Mots totaux: {result['total_words']}")

print("Décommenter et fournir un fichier de test")

## 3. Test de Génération de Script

In [ ]:
# Initialiser le générateur
script_gen = AudioScriptGenerator(os.getenv('GEMINI_API_KEY'), kb)

# Document de test (simulé)
test_doc = {
    'type': 'pdf',
    'metadata': {'title': 'Introduction aux Transformers'},
    'pages': [
        {'text': 'Les transformers sont une architecture révolutionnaire en NLP...', 'word_count': 500}
    ],
    'total_words': 500,
    'enhanced_metadata': {'main_topic': 'transformers'}
}

# Générer le script
script = script_gen.generate_script(test_doc, target_duration=180, style='conversational')

print("Script généré:")
print(f"\nIntro: {script['intro'][:150]}...")
print(f"\nQuiz: {len(script.get('quiz_questions', []))} questions")
if script.get('quiz_questions'):
    print(f"Question 1: {script['quiz_questions'][0]['question']}")

## 4. Test de Génération Audio

In [ ]:
# Initialiser le générateur audio
audio_gen = AudioGenerator()

# Générer l'audio
audio_path = audio_gen.generate_podcast(script)

print(f"Audio généré: {audio_path}")

# Afficher dans le notebook (si IPython)
from IPython.display import Audio
Audio(audio_path)

## 5. Test Complet du Pipeline

In [ ]:
def test_complete_pipeline(file_path):
    """Test du pipeline complet"""
    print("Test du pipeline complet\n")
    
    # 1. Traitement document
    print("Traitement du document...")
    doc_result = processor.process_document(file_path)
    print(f"   Type: {doc_result['type']}\n")
    
    # 2. Génération script
    print("Génération du script...")
    script = script_gen.generate_script(doc_result, target_duration=180)
    print(f"   {len(script.get('main_content', ''))} caractères\n")
    
    # 3. Génération audio
    print("Génération de l'audio...")
    audio_path = audio_gen.generate_podcast(script)
    print(f"   {audio_path}\n")
    
    # 4. Quiz
    print("Quiz généré:")
    for i, q in enumerate(script.get('quiz_questions', []), 1):
        print(f"   Q{i}: {q['question']}")
    
    return audio_path

# Tester avec votre fichier
# audio = test_complete_pipeline('../data/course_materials/m2/s2/week_1_intro.pdf')

print("Fournir un fichier de test pour exécuter")